In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
!export CUDA_VISIBLE_DEVICES=0

### Pretrain datasets

In [2]:
%%time

import tqdm, torch

df_trans = pq.read_table('data/trans_filtered_pretrained.parquet').to_pandas()

cols = ['url_host', 'request_cnt', 'part_of_day', 'event_time']

for col in tqdm.tqdm(cols):
    df_trans[col] = df_trans[col].apply(torch.tensor)


libgomp: Invalid value for environment variable OMP_NUM_THREADS
100%|██████████| 4/4 [00:11<00:00,  2.85s/it]

CPU times: user 22.4 s, sys: 14.1 s, total: 36.5 s
Wall time: 35.9 s


In [3]:
df_trans = df_trans.drop('price', axis =1)

In [4]:
from sklearn.model_selection import train_test_split

df_train_trans, df_valid_trans = train_test_split(df_trans, test_size = 0.1, random_state = 42)
df_train_trans = df_train_trans.to_dict(orient='records')
df_valid_trans = df_valid_trans.to_dict(orient='records')


libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [5]:
!export OMP_NUM_THREADS=16

In [6]:
len(df_train_trans), len(df_valid_trans)

(373785, 41532)

## Prepare embeddings

In [28]:
import pandas as pd
embedding_matrix = pd.read_pickle('artifacts/url_host_150.pickle').embeddings

In [29]:
url_embeddings = torch.nn.Embedding.from_pretrained(torch.Tensor(embedding_matrix), freeze=True)

## Train COLES

### Model definition

In [30]:
import torch
import pytorch_lightning as pl
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
import ptls

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'request_cnt': 'identity'
    },
    embeddings={
        #'price': {'in': 10, 'out': 2},
        #'region_name': {'in': 81, 'out': 4},
        #'city_name': {'in': 985, 'out': 16},
        #'cpe_manufacturer_name': {'in': 37, 'out': 4},
        #'cpe_model_name': {'in': 599, 'out': 16},
        #'cpe_type_cd': {'in': 4, 'out': 2}, 
        #'cpe_model_os_type': {'in': 3, 'out': 2}, 
        'part_of_day': {'in': 4, 'out': 1},
        'url_host': url_embeddings
    }
)


seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=1024,
    num_layers=2,
    type='gru',
)

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=10, gamma=0.9),
    head = torch.nn.Sequential(torch.nn.Linear(1024, 512), torch.nn.ReLU(), torch.nn.Linear(512, 512), 
            ptls.nn.Head(use_norm_encoder = True))
)

### Dataloader

In [10]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule


train_ds = MemoryMapDataset(data=df_train_trans)
valid_ds = MemoryMapDataset(data=df_valid_trans)

train_data=ColesDataset(train_ds, splitter = SampleSlices(split_count=5, cnt_min=20, cnt_max=200))
valid_data=ColesDataset(valid_ds, splitter = SampleSlices(split_count=5, cnt_min=20, cnt_max=200))

dl = PtlsDataModule(
    train_data=train_data, train_num_workers=16, train_batch_size=256, 
    valid_data=valid_data, valid_num_workers=16, valid_batch_size=256
)

### Trainer

In [11]:
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_epochs=50,
    limit_val_batches=100,
    gpus=[0],
    enable_progress_bar=False,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, dl)
print(trainer.logged_metrics)

logger.version = 11


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name               | Type            | Params
-------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0     
1 | _seq_encoder       | RnnSeqEncoder   | 39.9 M
2 | _validation_metric | BatchRecallTopK | 0     
3 | _head              | Sequential      | 787 K 
-------------------------------------------------------
10.7 M    Trainable params
30.0 M    Non-trainable params
40.7 M    Total params
162.634   Total estimated model params size (MB)


In [ ]:
torch.save(model.state_dict(), "coles-pretrained_150.pt")

# Inference

In [31]:
model.load_state_dict(torch.load("coles-pretrained_150.pt"))

<All keys matched successfully>

In [32]:
%%time

import tqdm, torch

df_trans = pq.read_table('data/trans_filtered_pretrained.parquet').to_pandas()

cols = ['url_host', 'request_cnt', 'part_of_day', 'event_time']

for col in tqdm.tqdm(cols):
    df_trans[col] = df_trans[col].apply(torch.tensor)

100%|██████████| 4/4 [00:12<00:00,  3.19s/it]

CPU times: user 24.7 s, sys: 14 s, total: 38.7 s
Wall time: 37.5 s


In [41]:
%%time
import tqdm
from ptls.data_load.datasets import inference_data_loader
import numpy as np

def pooling_inference(model, dl, device='cuda:0'):
    
    model.to(device)
    X = []
    for batch in tqdm.tqdm(dl):
        with torch.no_grad():
            x = model.seq_encoder.trx_encoder(batch.to(device)).payload
            out_max = torch.max(x, dim=1)[0]
            out_min = torch.min(x, dim=1)[0]
            out_mean = torch.mean(x, dim=1)
            out_std = torch.std(x, dim=1)
            features = torch.cat([out_max, out_min, out_mean, out_std], dim=1)      
            X += [features]
    return X

def embed_inference(model, dl, device='cuda:0'):
    
    model.to(device)
    X = []
    for batch in tqdm.tqdm(dl):
        with torch.no_grad():
            features = model.seq_encoder(batch.to(device))
            X += [features]
    return X

dl = inference_data_loader(df_trans.to_dict(orient='records'), num_workers=0, batch_size=32)
X_coles = torch.vstack(embed_inference(model, dl, )).cpu().numpy()
X_pool = torch.vstack(pooling_inference(model, dl, )).cpu().numpy()
X_embeds = np.concatenate([X_coles, X_pool], axis=1)


df_embeds = pd.DataFrame(X_embeds, columns=[f"embed_{e}" for e in range(X_embeds.shape[1])])
df_embeds['user_id'] = df_trans['user_id']

12979it [28:28,  7.60it/s]
12979it [06:47, 31.89it/s]


CPU times: user 6h 47min 27s, sys: 2min 22s, total: 6h 49min 49s
Wall time: 35min 25s


In [42]:
#df_embeds.to_csv('./data/coles_lstm.csv', index=False)

## Downstream

In [43]:
%%time

import bisect
import numpy as np

# df_embeds = pd.read_csv('./data/coles_lstm.csv')
df_public = pq.read_table('data/public_train.pqt').to_pandas().sort_values(by='user_id')

def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

y_age = df_public['age']
y_age = np.array(list(map(age_bucket, y_age)))
y_gender = np.array(df_public['is_male'])

X = df_public
X = X.merge(df_embeds, on="user_id", how='inner')
del X['user_id'], X['age'], X['is_male']

CPU times: user 2.81 s, sys: 567 ms, total: 3.38 s
Wall time: 3.37 s


## Gender

In [ ]:
# %%time
from catboost import CatBoostClassifier, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
not_na_gender = (y_gender != 'NA') & (y_gender != None)
x_train, x_test_gender, y_train, y_test_gender = train_test_split(X[not_na_gender], y_gender[not_na_gender], test_size = 0.1, random_state = 42)

clf_gender = CatBoostClassifier(
    iterations=1000,
    custom_metric=[metrics.AUC()],
    use_best_model=True,
    random_seed=42)
clf_gender.fit(x_train, y_train, metric_period=100, eval_set=(x_test_gender, y_test_gender))

Learning rate set to 0.122426
0:	learn: 0.6501417	test: 0.6502901	best: 0.6502901 (0)	total: 1.19s	remaining: 19m 48s
100:	learn: 0.4608559	test: 0.4659078	best: 0.4659078 (100)	total: 2m	remaining: 17m 53s
200:	learn: 0.4462947	test: 0.4581556	best: 0.4581556 (200)	total: 3m 59s	remaining: 15m 53s
300:	learn: 0.4358284	test: 0.4558160	best: 0.4558160 (300)	total: 5m 51s	remaining: 13m 37s
400:	learn: 0.4262897	test: 0.4541866	best: 0.4541866 (400)	total: 7m 47s	remaining: 11m 38s


In [ ]:
roc_auc_score(y_test_gender, clf_gender.predict_proba(x_test_gender)[:,1])

In [ ]:
print(f'GINI по полу {2 * roc_auc_score(y_test_gender, clf_gender.predict_proba(x_test_gender)[:,1]) - 1:2.3f}')

# Age

In [ ]:
%%time

from sklearn.metrics import classification_report

not_na_age = ~np.isnan(y_age)
x_train, x_test_age, y_train, y_test_age = train_test_split(X[not_na_age], y_age[not_na_age], test_size = 0.1, random_state = 42)

clf_age = CatBoostClassifier(iterations=1000,
    custom_metric=[metrics.Accuracy()],
    use_best_model=True,
    random_seed=42)
clf_age.fit(x_train, y_train, metric_period=100, eval_set=(x_test_age, y_test_age))

In [ ]:
print(classification_report(y_test_age, clf_age.predict(x_test_age), \
                            target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))

In [ ]:
0.765 + 2*0.46